In [1]:
# === CSV -> JSON with lon/lat (Colab-friendly) ===
# Edit these:
INPUT_CSV   = "C:\\Hackathon\\escape_2.csv"       # your uploaded CSV
OUTPUT_JSON = "C:\\Hackathon\\crowd_demo_escape_2_2.json"  # output file
CENTER_LAT  = 32.8419923                  # center latitude for meters/degree
NOTEBOOK_REF= "RobPy_Usage_Guide.ipynb"   # optional; used if present to pull the same function

import json, math
import pandas as pd
from pathlib import Path

def get_degree_coeffs(center_lat: float, nb_path: str | None = None):
    """Return (lat_coeff, lon_coeff) in meters/degree using RobPy-style formulas.
       If NOTEBOOK_REF defines degree_coefficients_fast, use it; else formula fallback."""
    def formula(lat_deg: float):
        φ = math.radians(lat_deg)
        lat_coeff = (111_132.92
                     - 559.82*math.cos(2*φ)
                     +   1.175*math.cos(4*φ)
                     -   0.0023*math.cos(6*φ))
        lon_coeff = (111_412.84*math.cos(φ)
                     -      93.5*math.cos(3*φ)
                     +       0.118*math.cos(5*φ))
        return float(lat_coeff), float(lon_coeff)

    nbp = Path(nb_path) if nb_path else None
    if nbp and nbp.exists():
        try:
            with open(nbp, "r", encoding="utf-8") as f:
                nb = json.load(f)
            code_cells = ["".join(c.get("source", [])) for c in nb.get("cells", []) if c.get("cell_type")=="code"]
            src = "\n\n".join(code_cells)
            ns = {}
            if "def degree_coefficients_fast" in src:
                exec(src, {}, ns)  # defines function in ns
                lat_c, lon_c = ns["degree_coefficients_fast"](center_lat)
                return float(lat_c), float(lon_c)
        except Exception as e:
            print(f"[Info] Could not load from notebook ({e}); using formula fallback.")
    return formula(center_lat)

def pick_col(df: pd.DataFrame, *names):
    m = {c.lower(): c for c in df.columns}
    for n in names:
        if n in m: return m[n]
    raise ValueError(f"Missing expected column. Looked for {names}. Found: {list(df.columns)}")

# 1) Read CSV
df = pd.read_csv(INPUT_CSV)

# 2) Identify columns (case-insensitive)
imei_col = pick_col(df, "imei", "phone_id", "phoneid", "device_id")
lat_col  = pick_col(df, "lat", "latitude")
lon_col  = pick_col(df, "lng", "lon", "longitude")
t_col    = pick_col(df, "timestamp", "t", "time")

# 3) Keep & rename
data = df[[imei_col, t_col, lat_col, lon_col]].copy()
data.columns = ["phone_id", "t", "lat", "lon"]

# 4) Ensure numeric
data["t"]   = pd.to_numeric(data["t"], errors="coerce")
data["lat"] = pd.to_numeric(data["lat"], errors="coerce")
data["lon"] = pd.to_numeric(data["lon"], errors="coerce")
data = data.dropna(subset=["lat", "lon"]).copy()

# 5) Coefficients for converting degree deltas to meters (for speed only)
lat_coeff, lon_coeff = get_degree_coeffs(CENTER_LAT, NOTEBOOK_REF if Path(NOTEBOOK_REF).exists() else None)

# 6) Compute speed per phone in m/s using lon/lat deltas -> meters
out = []
for pid, g in data.groupby("phone_id", sort=False):
    g = g.sort_values("t", kind="mergesort").reset_index(drop=True)

    g["dlat"] = g["lat"].diff()
    g["dlon"] = g["lon"].diff()
    g["dt"]   = g["t"].diff()

    # Euclidean distance in meters using meters/degree coefficients
    g["dist_m"] = ((g["dlat"]*lat_coeff)**2 + (g["dlon"]*lon_coeff)**2) ** 0.5

    def safe_speed(row):
        dt = row["dt"]
        if pd.isna(dt) or dt <= 0:
            return 0.0
        return float(row["dist_m"] / dt)

    g["speed"] = g.apply(safe_speed, axis=1)

    track = [
        {"t": (None if pd.isna(r.t) else float(r.t)),
         "lon": float(r.lon),
         "lat": float(r.lat),
         "speed": float(r.speed)}
        for _, r in g.iterrows()
    ]
    out.append({"phone_id": str(pid), "track": track})

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print("✓ Saved", OUTPUT_JSON)
print(f"lat_coeff = {lat_coeff:,.4f} m/deg | lon_coeff = {lon_coeff:,.4f} m/deg")


✓ Saved C:\Hackathon\crowd_demo_escape_2_2.json
lat_coeff = 110,901.6291 m/deg | lon_coeff = 93,619.4028 m/deg
